## 1. Setup and Dependencies

First, we clone the utility repository and install all necessary libraries, including the transformers library for BART and the SummerTime library required by the ResultsEval.py script.

In [1]:
# ==============================================================================
## 1. Setup, Dependencies, and Directory Change
# ==============================================================================
# Setup Dependencies
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install evaluate
!pip install accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=80acac36428b77c9dfa1471d6b88a3a756f52d2d73cab12874352c71179c3db6
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
import evaluate
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import os

# --- Configuration ---
MODEL_CHECKPOINT = "facebook/bart-base"
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128
OUTPUT_DIR = "./bart_meetingbank_results"
EVAL_BATCH_SIZE = 8
NUM_TRAIN_EPOCHS = 1 # Reduced for quick run. Increase for better results.

In [3]:
# Clone the utility repository (optional)
# !git clone https://github.com/YebowenHu/MeetingBank-utils.git

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to a clean output location
GDRIVE_ROOT =  "/content/drive/MyDrive/CS_685/youtube-video-summarization/"

%cd $GDRIVE_ROOT

Mounted at /content/drive
/content/drive/MyDrive/CS_685/youtube-video-summarization


## 2. Data Loading and Preprocessing for BART

We will use the Hugging Face datasets library for easy access to the MeetingBank data and the transformers library for preprocessing (tokenization) which is the core preprocessing step for models like BART.

In [4]:
# ==============================================================================
## 2. Configuration and Preprocessing
# ==============================================================================
print("\n--- Section 2: Data Loading and Preprocessing ---")

# 1. Load Data and Tokenizer
print("Loading dataset and tokenizer...")
raw_datasets = load_dataset("huuuyeah/meetingbank")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# 2. Define Preprocessing Function (Tokenization)
def preprocess_function(examples):
    # Tokenize input (transcript)
    model_inputs = tokenizer(
        examples["transcript"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"
    )

    # Tokenize label (summary)
    # The tokenizer's `as_target_tokenizer` context manager is used for label tokenization
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding="max_length"
        )

    # Replace padding token id with -100 for loss ignoring
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 3. Apply Preprocessing and Data Type Fix
print("Applying tokenization and data type fixes...")
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

# === Ensure all token ID arrays are standard integer types (int32) ===
# This prevents the OverflowError during prediction/decoding.
def ensure_int32(example):
    example["input_ids"] = np.array(example["input_ids"], dtype=np.int32)
    example["labels"] = np.array(example["labels"], dtype=np.int32)
    example["attention_mask"] = np.array(example["attention_mask"], dtype=np.int32)
    return example

tokenized_datasets = tokenized_datasets.map(ensure_int32)
tokenized_datasets.set_format("torch")
print("Preprocessing complete. Data types secured.")


--- Section 2: Data Loading and Preprocessing ---
Loading dataset and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.json:   0%|          | 0.00/88.4M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

...

Map:   0%|          | 0/862 [00:00<?, ? examples/s]

Map:   0%|          | 0/5169 [00:00<?, ? examples/s]

Map:   0%|          | 0/861 [00:00<?, ? examples/s]

Map:   0%|          | 0/862 [00:00<?, ? examples/s]

Preprocessing complete. Data types secured.


## 3. Training Setup and Fine-Tuning (Required to get a Fine-Tuned Model)

In [5]:
# ==============================================================================
## 3. Training Setup and Fine-Tuning
# ==============================================================================
print("\n--- Section 3: BART Fine-Tuning Setup ---")

# 1. Load the BART Model and Data Collator
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 2. Define Training Arguments (wandb disabled)
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),
    predict_with_generate=True,
    load_best_model_at_end=True,
    report_to="none", # FIX: Disable WANDB logging
)

# 3. Initialize Trainer (FutureWarning note: using processing_class for future compatibility)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer, # Preferred over passing 'tokenizer=tokenizer'
)

# 4. Start Fine-Tuning
print(f"Starting BART fine-tuning ({NUM_TRAIN_EPOCHS} epoch)...")
trainer.train()

# Find the path to the best saved model checkpoint
best_checkpoint_path = trainer.state.best_model_checkpoint if trainer.state.best_model_checkpoint else OUTPUT_DIR

print(f"\nBest/Final checkpoint path: {best_checkpoint_path}")


--- Section 3: BART Fine-Tuning Setup ---


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Starting BART fine-tuning (1 epoch)...


/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,1.860100,1.476759


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



Best/Final checkpoint path: ./bart_meetingbank_results/checkpoint-1293


## 4. Evaluation Function

In [6]:
# Load the ROUGE metric
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Ensure predictions are within valid token ID range and are integers
    max_vocab_id = tokenizer.vocab_size - 1
    preds = np.clip(preds, 0, max_vocab_id).astype(np.int32)

    # Decode predictions and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id).astype(np.int32)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE
    result = rouge_metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    # Format score
    result = {f"rouge_{k}": round(v * 100, 4) for k, v in result.items()}

    return result

## 5. Running Both Evaluations

In [7]:
# ==============================================================================
## 5. Running Both Evaluations
# ==============================================================================
print("\n--- Section 5: Running Evaluations ---")

# Define common evaluation arguments (wandb disabled)
eval_args = Seq2SeqTrainingArguments(
    output_dir="./evaluation_output",
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    predict_with_generate=True,
    disable_tqdm=False,
    report_to="none",
)

# --- A. Evaluation on the Pre-trained Model (Baseline) ---
print("\n" + "="*50)
print("Evaluating Pre-trained BART Model (Baseline)")
print("="*50)
pre_trained_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

pre_trained_trainer = Seq2SeqTrainer(
    model=pre_trained_model,
    args=eval_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

pre_trained_results = pre_trained_trainer.predict(
    test_dataset=tokenized_datasets["test"],
    # FIX: Pass generation kwargs directly (num_beams, max_length)
    num_beams=4,
    max_length=MAX_TARGET_LENGTH
)
print("Pre-trained Model ROUGE Results:")
pretrained_results = compute_metrics((pre_trained_results.predictions, pre_trained_results.label_ids))
# print(pretrained_results)

# Turn pretrained_results into dataframe
pretrained_results_df = pd.DataFrame(pretrained_results, index=[0])
print(pretrained_results_df)


--- Section 5: Running Evaluations ---

Evaluating Pre-trained BART Model (Baseline)


Pre-trained Model ROUGE Results:
{'rouge_rouge1': np.float64(35.8944), 'rouge_rouge2': np.float64(24.4663), 'rouge_rougeL': np.float64(31.0136), 'rouge_rougeLsum': np.float64(30.9908)}
   rouge_rouge1  rouge_rouge2  rouge_rougeL  rouge_rougeLsum
0       35.8944       24.4663       31.0136          30.9908


In [8]:
# --- B. Evaluation on the Fine-Tuned Model ---
print("\n" + "="*50)
print("Evaluating Fine-Tuned BART Model")
print("="*50)
# Load the best model found during training
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(best_checkpoint_path)

fine_tuned_trainer = Seq2SeqTrainer(
    model=fine_tuned_model,
    args=eval_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

fine_tuned_results = fine_tuned_trainer.predict(
    test_dataset=tokenized_datasets["test"],
    # FIX: Pass generation kwargs directly (num_beams, max_length)
    num_beams=4,
    max_length=MAX_TARGET_LENGTH
)
print("Fine-Tuned Model ROUGE Results:")
finetuned_results = compute_metrics((fine_tuned_results.predictions, fine_tuned_results.label_ids))
# print(finetuned_results)

# Turn fine-tuned results into dataframe
finetuned_results_df = pd.DataFrame(finetuned_results, index=[0])
print(finetuned_results_df)


Evaluating Fine-Tuned BART Model


Fine-Tuned Model ROUGE Results:
{'rouge_rouge1': np.float64(64.0704), 'rouge_rouge2': np.float64(54.198), 'rouge_rougeL': np.float64(61.5224), 'rouge_rougeLsum': np.float64(61.4926)}
   rouge_rouge1  rouge_rouge2  rouge_rougeL  rouge_rougeLsum
0       64.0704        54.198       61.5224          61.4926


## 6. Save the Evaluation Results

In [13]:
# ==============================================================================
## 6. Save Both Evaluation Results to CSV
# ==============================================================================
print("\n--- Section 6: Saving Predictions to CSV ---")

def save_predictions_to_csv(prediction_output, model_name, raw_data, tokenizer):
    """
    Decodes predictions, aligns them with inputs/references, and saves to CSV.
    """
    print(f"Saving predictions for {model_name}...")

    # 1. Decode Predictions (Generated Summaries)
    pred_ids = prediction_output.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]

    # Ensure predictions are within valid token ID range and are integers
    # Add clipping before casting to int32 to prevent OverflowError
    max_vocab_id = tokenizer.vocab_size - 1
    pred_ids = np.clip(pred_ids, 0, max_vocab_id).astype(np.int32)
    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    # 2. Decode Labels (Reference Summaries)
    label_ids = prediction_output.label_ids
    # Replace -100 padding with pad_token_id before decoding
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)

    # Cast to int32 before decoding
    label_ids = label_ids.astype(np.int32)
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # 3. Get Original Inputs
    original_inputs = raw_datasets["test"]["transcript"]

    # 4. Create DataFrame and Save
    results_df = pd.DataFrame({
        'Input_Transcript': original_inputs,
        'Reference_Summary': decoded_labels,
        'Predicted_Summary': decoded_preds
    })

    RESULTS_FILE_PATH = f"{model_name.lower().replace(' ', '_')}_predictions.csv"
    DIR_PATH = os.path.dirname('./data/meetingbank/')
    SAVE_PATH = os.path.join(DIR_PATH, RESULTS_FILE_PATH)
    results_df.to_csv(SAVE_PATH, index=False)

    print(f"Successfully saved to: {SAVE_PATH}")
    return SAVE_PATH

def save_evaluations_to_csv(results_df, model_name, base_dir='./data/meetingbank/'):
    """
    Saves a ROUGE score dataframe to a CSV file.
    """
    # Create directory if it doesn't exist
    os.makedirs(base_dir, exist_ok=True)

    # Sanitize model_name for filename
    sanitized_model_name = model_name.lower().replace(' ', '_').replace('-', '_')
    eval_path = os.path.join(base_dir, f"{sanitized_model_name}_rouge_scores.csv")
    results_df.to_csv(eval_path, index=False)
    print(f"Successfully saved {model_name} ROUGE scores to: {eval_path}")

# --- Execute Saving ---

# 1. Save Pre-trained Model Results
save_predictions_to_csv(
    pre_trained_results,
    "Pre-trained BART",
    raw_datasets,
    tokenizer
)

# 2. Save Fine-Tuned Model Results
save_predictions_to_csv(
    fine_tuned_results,
    "Fine-Tuned BART",
    raw_datasets,
    tokenizer
)

# 3. Save Pre-trained Evaluation Results (ROUGE scores)
save_evaluations_to_csv(pretrained_results_df, "Pre-trained BART")

# 4. Save Fine-tuned Evaluation Results (ROUGE scores)
save_evaluations_to_csv(finetuned_results_df, "Fine-Tuned BART")

print("\nAll prediction and evaluation files have been generated in the current directory.")


--- Section 6: Saving Predictions to CSV ---
Saving predictions for Pre-trained BART...
Successfully saved to: ./data/meetingbank/pre-trained_bart_predictions.csv
Saving predictions for Fine-Tuned BART...
Successfully saved to: ./data/meetingbank/fine-tuned_bart_predictions.csv
Successfully saved Pre-trained BART ROUGE scores to: ./data/meetingbank/pre_trained_bart_rouge_scores.csv
Successfully saved Fine-Tuned BART ROUGE scores to: ./data/meetingbank/fine_tuned_bart_rouge_scores.csv

All prediction and evaluation files have been generated in the current directory.
